In [2]:
import os
import sys

import numpy as np 
import open3d as o3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
demo_icp_clouds = o3d.data.DemoICPPointClouds()
src = o3d.io.read_point_cloud(demo_icp_clouds.paths[0])
dst = o3d.io.read_point_cloud(demo_icp_clouds.paths[1])

if not os.path.exists("./src.ply"): o3d.io.write_point_cloud("./src.ply", src)
if not os.path.exists("./dst.ply"): o3d.io.write_point_cloud("./dst.ply", dst)

P = np.asarray(src.points)
Q = np.asarray(dst.points)

print(P)
print(Q)
print(P.shape)
print(Q.shape)

[[1.01953125 0.88671875 2.27726722]
 [1.04296875 0.88671875 2.27699804]
 [1.05859375 0.88671875 2.27722216]
 ...
 [1.26609075 2.37890625 2.26171875]
 [1.26171875 2.38015151 2.26171875]
 [1.26171875 2.37890625 2.26367211]]
[[1.32421875 1.16796875 1.39391088]
 [1.10546875 1.18451285 1.33203125]
 [1.10546875 1.18359375 1.33258736]
 ...
 [2.76086402 2.58203125 1.14453125]
 [2.75390625 2.58381176 1.14453125]
 [2.75390625 2.58203125 1.15058577]]
(198835, 3)
(137833, 3)


In [80]:
# Kabsch algorithm for unique random pairs

print(P.shape[0])
N_all = P.shape[0] - P.shape[0]%P.shape[1]
print(N_all) # 0 ~ 198833
stride = int(N_all/3)
print(stride)

# unique random 3 indices
SEED = 0 # deterministic
indices_random = np.random.default_rng(seed=SEED).choice(P.shape[0], N_all, replace=False).reshape((stride, P.shape[1]))
print(indices_random)
print(indices_random.shape)

# select from indices
# P_selected = np.asmatrix(P[indices_random[0]])
P_selected = np.asmatrix(P[0])
print(P_selected)
print(type(P_selected))
# Q_selected = np.asmatrix(Q[indices_random[0]])
Q_selected = np.asmatrix(Q[0])
print(Q_selected)
print(type(Q_selected))

print(P_selected.shape)
print(Q_selected.shape)

198835
198834
66278
[[ 78262  67929 134603]
 [160826  67523 189929]
 [158760 124440 183588]
 ...
 [ 34847   3286  14959]
 [  8146  61206  53642]
 [101630 126649 169133]]
(66278, 3)
[[1.01953125 0.88671875 2.27726722]]
<class 'numpy.matrix'>
[[1.32421875 1.16796875 1.39391088]]
<class 'numpy.matrix'>
(1, 3)
(1, 3)


In [52]:
def rigid_transform_3D(A, B, scale):

    assert len(A) == len(B)
    N = A.shape[0]

    # mean of each point cloud
    Am = np.mean(A, axis=0)
    Bm = np.mean(B, axis=0)

    # if len(Am.shape) == 1: Am = Am.reshape(1, N)
    # if len(Bm.shape) == 1: Bm = Bm.reshape(1, N)

    # centered point clouds
    Ac = A - np.tile(Am, (N, 1))
    Bc = B - np.tile(Bm, (N, 1))

    H = np.transpose(Bc) * Ac # NOTE: Kabsch; H = P^T \cdot Q
    if scale: H /= N

    """
    Based on rotation formula, optimal rotation R is

    R = sqrt(H^T \cdot H) \cdot H^-1

    since directly solving this is complicated and hard to handle edge cases (e.g., singular H), use SVD
    """

    U, S, Vt = np.linalg.svd(H) # NOTE: Kabsch; H = USV^T
    
    R = Vt.T * U.T # NOTE: Kabsch; R = V \cdot U^T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("[DEBUG] Reflection detected")
        Vt[2, :] *= -1
        R = Vt.T * U.T
    
    if scale:
        Avar = np.var(A, axis=0).sum()
        c = 1 / (1 / Avar * np.sum(S)) # scale singular value
        t = -R.dot(Bm.T * c) + Am.T
        
    else:
        c = 1
        t = -R.dot(Bm.T) + Am.T

    return c, R, t


s, R, t = rigid_transform_3D(P_selected, Q_selected, False)
print(R)
print(t)

[[-0.23000691  0.57146007 -0.7877374 ]
 [ 0.01316091  0.81119175  0.58463213]
 [ 0.9731      0.12410209 -0.19410064]]
[[ 2.70830056]
 [-0.78519461]
 [ 0.00919052]]


In [57]:
P_selected2 = (R * Q_selected.T) + np.tile(t, (1, 3))
P_selected2 = P_selected2.T
print(P_selected)
print(P_selected2)

[[2.16015625 1.72265625 1.86595058]
 [2.17357731 1.60546875 1.85546875]
 [2.69921875 2.25390625 1.07963181]]
[[2.105586   1.81504416 1.93535721]
 [2.31675078 1.6321899  1.65460148]
 [2.61061553 2.13479719 1.21109245]]


In [56]:
# export point clouds

np.savetxt("P_selected.txt", P_selected)
np.savetxt("Q_selected.txt", Q_selected)

In [79]:
# TODO: check ordering

T = np.loadtxt("reg_p2l.txt")

print(Q_selected[0])
print(Q_selected[0].shape)

q_h = np.ndarray((Q_selected[0].shape[0], Q_selected[0].shape[1]+1))
q_h[:, :3] = Q_selected[0]
q_h[:, 3] = 1.0
q_h = q_h.T
print(q_h)
print(q_h.shape)

Tq = np.linalg.inv(T).dot(q_h).T[:, :3]
print(Tq)
print(P_selected[0])

[[2.04720283 2.00390625 1.62109375]]
(1, 3)
[[2.04720283]
 [2.00390625]
 [1.62109375]
 [1.        ]]
(4, 1)
[[2.62225683 1.97586039 1.5034477 ]]
[[2.16015625 1.72265625 1.86595058]]
